<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Признаки
RowNumber — индекс строки в данных
CustomerId — уникальный идентификатор клиента
Surname — фамилия
CreditScore — кредитный рейтинг
Geography — страна проживания
Gender — пол
Age — возраст
Tenure — сколько лет человек является клиентом банка
Balance — баланс на счёте
NumOfProducts — количество продуктов банка, используемых клиентом
HasCrCard — наличие кредитной карты
IsActiveMember — активность клиента
EstimatedSalary — предполагаемая зарплата
Целевой признак
Exited — факт ухода клиента

## Подготовка данных

In [ ]:
#подключим библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, r2_score
from sklearn.utils import shuffle
from matplotlib import pyplot
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [3]:
#выведем на экран данные
df = pd.read_csv('./Churn.csv')
display(df.sample(10))
df.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9116,9117,15672082,Schatz,562,France,Male,62,3.0,0.00,2,1,0,105986.01,0
9474,9475,15631481,Thomson,673,France,Male,51,8.0,79563.36,2,1,1,172200.91,0
6287,6288,15668289,McWilliams,690,Spain,Male,32,2.0,76087.98,1,0,1,151822.66,0
5890,5891,15712777,Kao,482,France,Male,38,NaN,124976.19,1,1,0,35848.12,0
6178,6179,15746986,Howe,850,Germany,Female,40,4.0,97990.49,2,0,0,106691.02,0
3026,3027,15722758,Allan,585,France,Male,40,NaN,0.00,2,0,0,146156.98,0
8621,8622,15673820,Woodward,568,France,Male,33,7.0,0.00,2,1,0,143450.61,0
2286,2287,15640161,Calabrese,618,Germany,Male,44,5.0,157955.83,2,0,0,139297.71,0
6066,6067,15580249,Lori,502,France,Male,45,0.0,0.00,1,0,0,84663.21,0
4188,4189,15786153,McKenzie,644,Germany,Male,47,9.0,137774.11,2,1,0,151902.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [ ]:
#удалим лишние столбцы,не несущие необходимой информ.
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [ ]:
data = (np.random.randint(0,10,size=df['Tenure'].isna().sum()))
df.loc[df['Tenure'].isna(),'Tenure']= data

In [ ]:
#встречаются категориальные признаки,преобразуем эти столбцы.
df = pd.get_dummies(df, drop_first=True)
df.sample(5)

In [ ]:
display(df.sample(10))
df.info()

In [ ]:
#df.isna().sum()
df.info()

## Исследование задачи

In [ ]:
#Разделим данные
features = df.drop('Exited', axis=1)
target = df['Exited']



In [ ]:
#Разделим получившиеся выборки на обучающую, валидационную, тестовую.
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

print('Размеры наборов данных:')
print('Обучающий:    ', features_train.shape)
print('Валидационный:', features_valid.shape)
print('Тестовый:     ', features_test.shape)


In [ ]:
#Маштабируем числовые признаки

numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [ ]:
#Дерево решений
def decision_tree(features_tr, target_tr, features_val, target_val, cw=None):
    best_depth = 0
    best_f1 = 0
    best_roc_auc = 0
    best_predictions = 0
    
    for depth in range(2, 31):
    
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight=cw)
        model.fit(features_tr, target_tr)
    
        predictions_valid = model.predict(features_val)
        f1 = f1_score(target_val, predictions_valid)
        
        predictions = model.predict_proba(features_val)[:,1]
        roc_auc = roc_auc_score(target_val, predictions)
        
        
        if f1 > best_f1:
            best_depth = depth
            best_f1 = f1    
            best_roc_auc = roc_auc
            best_predictions = predictions_valid 
            
    print("Глубина дерева:", best_depth)
    print("f1-мера:       ", best_f1)
    print("roc_auc:       ", best_roc_auc)
    print('Доля правильных ответов:',accuracy_score(target_val, best_predictions))
    print('Соотношение классов:',pd.Series(best_predictions).value_counts(normalize=True), sep='\n')
decision_tree(features_train, target_train, features_valid, target_valid)

In [ ]:
#Random Forest
def random_forest(features_tr, target_tr, features_val, target_val, cw=None):
    best_est = 0
    best_depth = 0
    best_f1 = 0
    best_roc_auc = 0
    best_predictions = 0

    for est in range(2, 21):
        for depth in range(2,25):
        
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight=cw)
    
            model.fit(features_tr, target_tr)
    
            predictions_valid = model.predict(features_val)
            f1 = f1_score(target_val, predictions_valid)
            
            predictions =model.predict_proba(features_val)[:,1]
            roc_auc = roc_auc_score(target_val, predictions)
        
            if f1 > best_f1:
                best_est = est
                best_depth = depth
                best_f1 = f1    
                best_roc_auc = roc_auc  
                best_predictions = predictions_valid 
    
    print("Кол-во деревьев:", best_est)
    print("Глубина дерева: ", best_depth)
    print("f1-мера:        ", best_f1)
    print("roc_auc:        ", best_roc_auc)
    print('Доля правильных ответов:',accuracy_score(target_val, best_predictions))
    print('Соотношение классов:',pd.Series(best_predictions).value_counts(normalize=True), sep='\n')

random_forest(features_train, target_train, features_valid, target_valid)

In [ ]:
#LogisticRegression
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
    
predictions_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predictions_valid)

predictions = model.predict_proba(features_valid)[:,1]
roc_auc = roc_auc_score(target_valid, predictions)

print("f1-мера: ", f1)
print("roc_auc: ", roc_auc)  
print('Доля правильных ответов:',accuracy_score(target_valid, predictions_valid))
print('Соотношение классов:',pd.Series(predictions_valid).value_counts(normalize=True), sep='\n')



In [ ]:
# Проверим, как часто в целевом признаке встречается класс «1» или «0»
class_frequency = target.value_counts(normalize=True)
print(class_frequency)
class_frequency.plot(kind='bar') 

Вывод: Лучше всего себя показали дерево решений и случайный лес. 

## Борьба с дисбалансом

Взвешивние классов. 

In [ ]:
#Дерево решений
decision_tree(features_train, target_train, features_valid, target_valid, cw ='balanced')

In [ ]:
#Случайный лес
random_forest(features_train, target_train, features_valid, target_valid, cw='balanced')

Показатели f1 и roc_auc возрасли.

Напишем функцию уменьшения и увеличения выборок. 

In [ ]:
# Создадим функцию для увеличения обучающей выборки класса 1 в несколько раз
def upsample(features, target, repeat):
    # Разделилим обучающую выборку на отрицательные и положительные объекты.
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    # Объединим таблицы
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    # Перемешаем данные
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
# Применим функцию для увеличения выборки класса 1 в 4 раза
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
#Модель дерева решений с учетом балансировки

best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
best_predicted_valid = 0
best_roc_auc = 0
for depth in range(1, 51):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # инициализируем модель DecisionTreeClassifier с параметром random_state=12345 и max_depth=depth
    model.fit(features_upsampled, target_upsampled) # обучим модель на новой тренировочной выборке
    predicted_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    
    f1 = f1_score(target_valid, predicted_valid)
    
    predictions = model.predict_proba(features_valid)[:,1]
    roc_auc = roc_auc_score(target_valid, predictions)
    
    
    if f1 > best_tree_result:
        best_tree_model = model
        best_tree_result = f1
        best_tree_depth = depth
        best_predicted_valid = predicted_valid
        best_roc_auc = roc_auc
        
print("F1-мера наилучшей модели на валидационной выборке:", best_tree_result,
      "Глубина дерева:", best_tree_depth,"roc_auc:", best_roc_auc, sep='\n')

In [ ]:
#Модель Случайный лес с учетом балансировки
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
best_predicted_valid = 0
best_roc_auc = 0
for est in range(21, 41, 2):
    for depth in range (31, 51, 2):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        # инициализируем модель RandomForestClassifier с параметрами random_state=12345, 
        # n_estimators=est и max_depth=depth
        model.fit(features_upsampled, target_upsampled) # обучим модель на тренировочной выборке
        predicted_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
        f1 = f1_score(target_valid, predicted_valid)
        
        predictions = model.predict_proba(features_valid)[:,1]
        roc_auc = roc_auc_score(target_valid, predictions)
        
        
        if f1 > best_forest_result:
            best_forest_model = model
            best_forest_result = f1
            best_forest_est = est
            best_forest_depth = depth
            best_predicted_valid = predicted_valid
            best_roc_auc = roc_auc
            

print("F1-мера наилучшей модели на валидационной выборке:", best_forest_result,
      "Количество деревьев:", best_forest_est, "Максимальная глубина:", best_forest_depth,"roc_auc:", best_roc_auc, sep='\n')

In [ ]:
# Применим функцию для увеличения выборки класса 1 в 4 раза
features_downsample, target_downsample = upsample(features_train, target_train, 4)

In [ ]:
# Создадим функцию для уменьшения обучающей выборки класса 1 в несколько раз
def downsample(features, target, repeat):
    # Разделилим обучающую выборку на отрицательные и положительные объекты.
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    # Объединим таблицы
    features_downsample = pd.concat([features_zeros] + [features_ones] * repeat)
    target_downsample = pd.concat([target_zeros] + [target_ones] * repeat)
    # Перемешаем данные
    features_downsample, target_downsample = shuffle(
        features_downsample, target_downsample, random_state=12345)
    
    return features_downsample, target_downsample

In [ ]:
#Модель дерева решений с учетом балансировки

best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
best_predicted_valid = 0
best_roc_auc = 0
for depth in range(1, 51):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # инициализируем модель DecisionTreeClassifier с параметром random_state=12345 и max_depth=depth
    model.fit(features_downsample, target_downsample) # обучим модель на новой тренировочной выборке
    predicted_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    f1 = f1_score(target_valid, predicted_valid)
    
    predictions = model.predict_proba(features_valid)[:,1]
    roc_auc = roc_auc_score(target_valid, predictions)
    
    
    if f1 > best_tree_result:
        best_tree_model = model
        best_tree_result = f1
        best_tree_depth = depth
        best_predicted_valid = predicted_valid
        best_roc_auc = roc_auc
        

print("F1-мера наилучшей модели на валидационной выборке:", best_tree_result,
      "Глубина дерева:", best_tree_depth,"roc_auc:", best_roc_auc, sep='\n')

In [ ]:
#Модель Случайный лес с учетом балансировки
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
best_predicted_valid = 0
best_roc_auc = 0
for est in range(21, 41, 2):
    for depth in range (31, 51, 2):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        # инициализируем модель RandomForestClassifier с параметрами random_state=12345, 
        # n_estimators=est и max_depth=depth
        model.fit(features_downsample, target_downsample) # обучим модель на тренировочной выборке
        predicted_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
        f1 = f1_score(target_valid, predicted_valid)
        
        
        predictions = model.predict_proba(features_valid)[:,1]
        roc_auc = roc_auc_score(target_valid, predictions)
        
        if f1 > best_forest_result:
            best_forest_model = model
            best_forest_result = f1
            best_forest_est = est
            best_forest_depth = depth
            best_predicted_valid = predicted_valid
            best_roc_auc = roc_auc

print("F1-мера наилучшей модели на валидационной выборке:", best_forest_result,
      "Количество деревьев:", best_forest_est, "Максимальная глубина:", best_forest_depth, "roc_auc:", best_roc_auc, sep='\n')

## Тестирование модели

Финанльное теслирование

In [ ]:
#Объеденим выборки для тестирования
features = pd.concat([features_train, features_valid], ignore_index=True)
target = pd.concat([target_train, target_valid], ignore_index=True)

In [ ]:

model = RandomForestClassifier(random_state=12345, n_estimators=20,max_depth=8)

model.fit(features_downsample, target_downsample)
predictions = model.predict(features_test)

f1 = f1_score(target_test, predictions)

predictions = model.predict_proba(features_test)[:,1]
roc_auc = roc_auc_score(target_test, predictions)

print(' f1 = {}, \n roc_auc = {}'.format(round(f1,4), round(roc_auc,4)))
print('')

importance = model.feature_importances_

for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

pyplot.bar([x for x in range(len(importance))], importance)
plt.title('Важность факторов')
pyplot.show()

n=1


Вывод:

Модели продемонстрировали хороший результат на тестовой выборке,
у обоих моделей f1-мера больше 0.59, согласно условиям задачи. auc_roc > 0.5, модели адекватны.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*